In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [6]:
sub = 'sub-04'
ses = 'ses-01'

battery = 'camcan'
# task_source = 'gng'
task = 'stop-nogo'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery, ses)
os.chdir(datadir)
csv_files = sorted(glob.glob(task + '*.txt'))
tsv_files = []
i = 1
# for i in range(len(csv_files)):
tsv_files.append(task + '_' + sub + '_run-0' + str(i+1) + '.tsv')

In [3]:
csv_files

['stop-nogo_sub-11_run-01_9221-1530.txt',
 'stop-nogo_sub-11_run-02_9221-1543.txt']

In [9]:
tsv_files

['stop-nogo_sub-05_run-02.tsv']

In [7]:
# Read in the rest of the file
file_df = pd.read_csv(os.path.join(datadir, csv_files[i]), sep='\t', skipinitialspace=True)

In [12]:
file_df

,tr#,type,arrLR,Stype,Sdel,butt,resp,resp2,RT,RT2,RTsum,eval,relStart
0,1,0,1,0,0,1,1,0,659,0,0,1,0
1,2,0,2,0,0,2,2,0,432,0,0,1,2524
2,3,1,2,3,300,2,0,2,0,256,556,2,5066
3,4,0,1,0,0,1,1,0,323,0,0,1,7649
4,5,1,2,2,250,2,0,0,0,0,0,1,10190
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,236,0,1,0,0,1,1,0,391,0,0,1,597754
236,237,0,2,0,0,2,2,0,349,0,0,1,600280
237,238,0,2,0,0,2,2,0,449,0,0,1,602822
238,239,1,2,3,150,2,0,0,0,0,0,1,605346


In [29]:
file_df.keys()

Index(['tr#', 'type', 'arrLR', 'Stype', 'Sdel', 'butt', 'resp', 'resp2', 'RT',
       'RT2', 'RTsum', 'eval', 'relStart'],
      dtype='object')

In [8]:
# Extract the onsets of stimulus, probe and response
trial_onsets = (file_df['relStart'] + 500)

# Find the 4 trials types:
# 1. Go trials ('type' = 0)
# 2. No-go trials ('type' = 1 and 'Stype' = 1)
# 3. Successful stop trials ('type' = 1 and 'Stype' = 2 or 3 and RT1 = 0 and RT2 = 0)
# 3. Unsuccessful stop trials ('type' = 1 and 'Stype' = 2 or 3 and (RT1 != 0 or RT2 != 0))
go_trials = file_df['type'] == 0
unsucgo_trials = file_df.loc[go_trials][file_df['RT'] == 0].index
nogo_trials = (file_df['type'] == 1) & (file_df['Stype'] == 1)
unsucnogo_trials = file_df.loc[nogo_trials][file_df['RT'] != 0].index
sucstop_trials = (file_df['type'] == 1) & ((file_df['Stype'] == 2) | (file_df['Stype'] == 3)) & \
            (file_df['RT2'] == 0)
unsucstop_trials = (file_df['type'] == 1) & ((file_df['Stype'] == 2) | (file_df['Stype'] == 3)) & \
            (file_df['RT2'] != 0)

# Calculate trial durations
durations = np.array(np.ones(len(trial_onsets)), dtype=np.int_)
durations[go_trials] = file_df.loc[go_trials]['RT']
durations[unsucgo_trials] = 2500
durations[nogo_trials] = 2500
durations[unsucnogo_trials] = file_df.loc[unsucnogo_trials]['RT']
durations[sucstop_trials] = 2500
durations[unsucstop_trials] = file_df.loc[unsucstop_trials]['RTsum']

# Assign trial types
trial_types = np.array(np.ones(len(trial_onsets)), dtype=np.object_)
trial_types[go_trials] = 'go'
trial_types[nogo_trials] = 'nogo'
trial_types[sucstop_trials] = 'successful_stop'
trial_types[unsucstop_trials] = 'unsuccessful_stop'

/home/sshankar/env/fastsrm/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
/home/sshankar/env/fastsrm/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


In [8]:
trial_types

array(['go', 'go', 'unsuccessful_stop', 'go', 'unsuccessful_stop', 'go',
       'go', 'go', 'go', 'go', 'go', 'go', 'go', 'go', 'go', 'go', 'go',
       'go', 'go', 'unsuccessful_stop', 'go', 'unsuccessful_stop', 'go',
       'go', 'go', 'unsuccessful_stop', 'nogo', 'go', 'go',
       'unsuccessful_stop', 'nogo', 'nogo', 'nogo', 'go', 'go', 'go',
       'go', 'nogo', 'nogo', 'go', 'go', 'go', 'unsuccessful_stop', 'go',
       'go', 'unsuccessful_stop', 'nogo', 'unsuccessful_stop',
       'successful_stop', 'go', 'nogo', 'go', 'successful_stop', 'nogo',
       'go', 'go', 'go', 'go', 'go', 'nogo', 'go', 'unsuccessful_stop',
       'go', 'go', 'nogo', 'go', 'go', 'go', 'go', 'go', 'go', 'go',
       'unsuccessful_stop', 'go', 'go', 'go', 'go', 'go', 'go', 'go',
       'go', 'go', 'go', 'go', 'go', 'go', 'go', 'unsuccessful_stop',
       'nogo', 'go', 'successful_stop', 'go', 'unsuccessful_stop', 'go',
       'go', 'go', 'go', 'go', 'go', 'go', 'go', 'unsuccessful_stop',
       'unsuccess

In [9]:
trial_onsets

0         500
1        3041
2        5567
3        8150
4       10693
        ...  
235    598438
236    600965
237    603489
238    606014
239    608563
Name: relStart, Length: 240, dtype: int64

In [13]:
durations

array([2500,  479,  429,  372,  311,  371,  412,  348,  346,  774,  377,
       2500,  373,  293,  414,  383,  410,  365,  345,  356, 2500,  393,
        392,  392,  327,  334, 2500,  428,  340,  316,  373,  361,  305,
        303,  349,  316,  361,  374,  317,  313,  618,  495,  323,  262,
       2500,  364,  365,  436,  305,  341,  338,  474, 2500, 2500, 2500,
        425,  554,  289,  432,  338,  422,  334,  377, 2500,  362,  332,
        448,  469, 2500,  447,  359,  545,  305,  364,  338,  393,  323,
        526,  264,  286,  389,  340,  341,  345,  362,  283, 2500,  303,
        313,  356,  316, 2500,  369, 2500,  472,  298,  477,  363,  322,
       2500,  426,  356,  309,  519,  292,  362, 2500,  490,  347,  596,
        312,  371,  357, 2500,  331, 2500,  443,  341,  339,  329,  453,
        379,  301,  325, 2500,  440,  395,  420,  428,  419,  422,  348,
        345,  402, 2500, 2500,  418,  338,  438, 2500, 2500, 2500, 2500,
        374,  429,  745,  359,  445,  325,  439,  3

In [9]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':trial_onsets/1000, 'duration':durations/1000, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_files[0]), sep='\t', float_format='%0.3f', index=False)